# AWS Bedrock RAG Infrastructure Setup

This notebook guides you through creating AWS infrastructure for a Bedrock RAG implementation.
**VPC creation is handled by an external script (`create_vpc_script.py`).**
Subsequent resources (Aurora, S3, Bedrock KB) are created step-by-step in this notebook.

# 1. Imports
Import necessary libraries

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
import time

# 2. Configuration
Define configuration variables for the infrastructure.

In [ ]:
AGENT_NAME = "AppDemoAgentWKB"  # Choose a unique name for your agent
KNOWLEDGE_BASE_ID = "<YOUR_EXISTING_KNOWLEDGE_BASE_ID>" # e.g., "ABCDEFGHIJ"
KNOWLEDGE_BASE_ARN = "<YOUR_EXISTING_KNOWLEDGE_BASE_ARN>" # e.g., "arn:aws:bedrock:us-east-1:123456789012:knowledge-base/ABCDEFGHIJ"
AGENT_EXECUTION_ROLE_ARN = "<YOUR_AGENT_IAM_ROLE_ARN>" # e.g., "arn:aws:iam::123456789012:role/MyBedrockAgentRole"
FOUNDATION_MODEL_ARN = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2" # Example: Change to your desired model/region

# ** OPTIONAL: Customize these values **
AGENT_DESCRIPTION = "Agent specialized in construction and heavy machines with access to knowledgebase and data in relationaldatabase"
AGENT_INSTRUCTION = """
You are a helpful customer support agent.
Your goal is to answer customer questions accurately based *only* on the information provided in the attached knowledge base.
If the knowledge base does not contain the answer, clearly state that the information is not available in the provided documents.
Do not make up answers or use external knowledge.
Be polite and concise.
"""
IDLE_SESSION_TTL_SECONDS = 1800  # 30 minutes timeout
AWS_REGION = "us-east-1" # Change if your KB/resources are in a different region


In [ ]:
try:
    bedrock_agent_client = boto3.client('bedrock-agent', region_name=AWS_REGION)
except Exception as e:
    print(f"Error creating Boto3 client: {e}")
    exit()

In [ ]:
knowledge_base_configuration = [
    {
        'knowledgeBaseId': KNOWLEDGE_BASE_ID,
        'description': "Use this knowledge base to find information about product features, setup, and troubleshooting."
        # Note: Retrieval configuration is set on the KB itself,
        # but you can add specific instructions here.
    }
    # Add more dictionaries here if connecting multiple KBs
]

In [ ]:
LAMBDA_FUNCTION_ARN = "<YOUR_LAMBDA_ARN>"

In [ ]:
OPENAPI_SCHEMA = {
    "openapi": "3.0.0",
    "info": {
        "title": "Wikipedia Finder API",
        "version": "1.0.0"
    },
    "paths": {
        "/wikipedia": {
            "post": {
                "summary": "Get Wikipedia URL for celebrity",
                "parameters": [
                    {
                        "name": "celebrity_name",
                        "in": "query",
                        "required": True,
                        "schema": {"type": "string"}
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Success response",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "url": {"type": "string"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

In [ ]:
ACTION_GROUP_CONFIG = [
    {
        'actionGroupName': 'WikipediaAPI',
        'description': 'Fetches Wikipedia URLs using OpenAPI schema',
        'actionGroupExecutor': {
            'lambda': LAMBDA_FUNCTION_ARN
        },
        'apiSchema': {
            'payload': json.dumps(OPENAPI_SCHEMA)  # Use S3 reference for large schemas
        }
    }
]

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=AGENT_NAME,
    agentResourceRoleArn=AGENT_EXECUTION_ROLE_ARN,
    foundationModel=FOUNDATION_MODEL_ARN,
    instruction=AGENT_INSTRUCTION,
    description=AGENT_DESCRIPTION,
    idleSessionTTLInSeconds=IDLE_SESSION_TTL_SECONDS,
    knowledgeBases=knowledge_base_configuration,
    actionGroups=ACTION_GROUP_CONFIG  # Added action groups
)

agent_details = response.get('agent', {})
agent_id = agent_details.get('agentId')
agent_arn = agent_details.get('agentArn')
agent_status = agent_details.get('agentStatus')

print("\n--- Agent Creation Initiated ---")
print(f"Agent Name: {agent_details.get('agentName')}")
print(f"Agent ID: {agent_id}")
print(f"Agent ARN: {agent_arn}")
print(f"Initial Status: {agent_status}")
print("------------------------------")
print("\nAgent creation can take a few minutes.")
print("Check the AWS Bedrock console for the current status.")

In [ ]:
while agent_status not in ['PREPARED', 'FAILED']:
    print(f"Current agent status: {agent_status}. Waiting...")
    time.sleep(30) # Check every 30 seconds
    get_agent_response = bedrock_agent_client.get_agent(agentId=agent_id)
    agent_status = get_agent_response.get('agent', {}).get('agentStatus')
    
    if agent_status == 'PREPARED':
        print(f"\nAgent '{agent_id}' is now PREPARED and ready to use.")
    elif agent_status == 'FAILED':
        print(f"\nAgent '{agent_id}' FAILED preparation. Check console for details.")

